In [1]:
import pandas as pd
import numpy as np
#import seaborn as sns #For plotting
from sklearn.model_selection import GroupShuffleSplit 
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTENC

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('data/grouped_data.csv')

In [32]:
df.head(5)

,gene_id,transcript_id,transcript_position,relative_position,fivemers,label,dwelling_time_min,dwelling_time_max,dwelling_time_mean,dwelling_time_median,...,mean_current_skew,order_1,order_2,order_3,order_4,order_5,count_A,count_C,count_G,count_T
0,ENSG00000000003,ENST00000373020,511,1,ATAAC,0,0.00266,0.0169,0.007247,0.005990,...,0.591063,A,T,A,A,C,3,1,0,1
1,ENSG00000000003,ENST00000373020,512,2,TAACT,0,0.00232,0.0461,0.011177,0.007735,...,-0.177996,T,A,A,C,T,2,1,0,2
2,ENSG00000000003,ENST00000373020,513,3,AACTC,0,0.00266,0.0219,0.008500,0.006870,...,-3.353925,A,A,C,T,C,2,2,0,1
3,ENSG00000000003,ENST00000373020,688,1,TAAAC,0,0.00232,0.0279,0.009868,0.007640,...,-1.322034,T,A,A,A,C,3,1,0,1
4,ENSG00000000003,ENST00000373020,689,2,AAACA,0,0.00232,0.0179,0.007332,0.006420,...,-0.931646,A,A,A,C,A,4,1,0,0


In [28]:
len(df['fivemers'].unique())

66

In [ ]:
pd.get_dummies(df, columns=['fivemers', 'order_1', 'order_2', 'order_3', 'order_4', 'order_5', ])

In [33]:
df.shape

(365514, 33)

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365514 entries, 0 to 365513
Data columns (total 33 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   gene_id               365514 non-null  object 
 1   transcript_id         365514 non-null  object 
 2   transcript_position   365514 non-null  int64  
 3   relative_position     365514 non-null  int64  
 4   fivemers              365514 non-null  object 
 5   label                 365514 non-null  int64  
 6   dwelling_time_min     365514 non-null  float64
 7   dwelling_time_max     365514 non-null  float64
 8   dwelling_time_mean    365514 non-null  float64
 9   dwelling_time_median  365514 non-null  float64
 10  dwelling_time_std     365514 non-null  float64
 11  dwelling_time_skew    365514 non-null  float64
 12  sd_current_min        365514 non-null  float64
 13  sd_current_max        365514 non-null  float64
 14  sd_current_mean       365514 non-null  float64
 15  

In [8]:
df.describe()

,transcript_position,relative_position,label,dwelling_time_min,dwelling_time_max,dwelling_time_mean,dwelling_time_median,dwelling_time_std,dwelling_time_skew,sd_current_min,...,mean_current_min,mean_current_max,mean_current_mean,mean_current_median,mean_current_std,mean_current_skew,count_A,count_C,count_G,count_T
count,365514.000000,365514.000000,365514.000000,365514.000000,365514.000000,365514.000000,365514.000000,365514.000000,365514.000000,365514.000000,...,365514.000000,365514.000000,365514.000000,365514.000000,365514.000000,365514.000000,365514.000000,365514.000000,365514.000000,365514.000000
mean,1615.868407,2.000000,0.014979,0.002345,0.024947,0.007764,0.006592,0.004625,1.579478,1.611517,...,94.580259,109.671788,102.646318,102.752992,3.026898,-0.210321,2.169569,1.332548,0.921341,0.576542
std,1395.133950,0.816498,0.121468,0.000478,0.009412,0.001682,0.001541,0.001442,0.697225,0.805870,...,14.007864,16.032900,15.412760,15.577274,1.184796,0.907801,0.880328,0.524325,0.736789,0.648432
min,2.000000,1.000000,0.000000,0.001660,0.005980,0.003537,0.002660,0.000922,-0.617264,0.044000,...,61.000000,76.200000,72.231429,71.700000,0.531099,-5.269245,1.000000,1.000000,0.000000,0.000000
25%,673.000000,1.000000,0.000000,0.001990,0.018300,0.006555,0.005520,0.003600,1.111282,1.070000,...,83.100000,95.600000,89.258106,89.250000,2.173622,-0.774930,2.000000,1.000000,0.000000,0.000000
50%,1256.000000,2.000000,0.000000,0.002320,0.023200,0.007435,0.006310,0.004399,1.495026,1.400000,...,92.500000,108.000000,100.339898,100.000000,2.790958,-0.193065,2.000000,1.000000,1.000000,0.000000
75%,2139.000000,3.000000,0.000000,0.002480,0.029600,0.008651,0.007310,0.005412,1.939889,1.930000,...,106.000000,125.000000,117.654371,118.000000,3.648482,0.356033,3.000000,2.000000,1.000000,1.000000
max,18731.000000,3.000000,1.000000,0.009890,0.138000,0.021581,0.017900,0.020858,8.180598,8.230000,...,132.000000,156.000000,137.120000,137.000000,19.888538,9.193026,4.000000,3.000000,3.000000,2.000000


In [35]:
#Get non-numerical features.
non_num_cols = list(set(df.columns) - set(df.describe().columns))
print('Non-numerical features: ',non_num_cols)

Non-numerical features:  ['fivemers', 'order_1', 'order_4', 'transcript_id', 'gene_id', 'order_2', 'order_3', 'order_5']


In [36]:
# create list with categorical features' names
nonum_cols_names = ['order_4', 'order_2', 'order_1', 'fivemers', 'order_3', 'order_5', 'relative_position']

# create list with numerical features' names
num_cols_names = ['dwelling_time_min', 'dwelling_time_max', 'dwelling_time_mean', 'dwelling_time_median', 'dwelling_time_std', 
                  'dwelling_time_skew', 'sd_current_min', 'sd_current_max', 'sd_current_mean', 'sd_current_median', 'sd_current_std',
                  'sd_current_skew', 'mean_current_min', 'mean_current_max', 'mean_current_mean', 'mean_current_median', 'mean_current_std',
                  'mean_current_skew', 'count_A', 'count_C', 'count_G', 'count_T']

In [59]:
#Assign appropriate data type to categorical variables.
df[nonum_cols_names] = df[nonum_cols_names].astype('category')

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365514 entries, 0 to 365513
Data columns (total 33 columns):
 #   Column                Non-Null Count   Dtype   
---  ------                --------------   -----   
 0   gene_id               365514 non-null  object  
 1   transcript_id         365514 non-null  object  
 2   transcript_position   365514 non-null  int64   
 3   relative_position     365514 non-null  category
 4   fivemers              365514 non-null  category
 5   label                 365514 non-null  int64   
 6   dwelling_time_min     365514 non-null  float64 
 7   dwelling_time_max     365514 non-null  float64 
 8   dwelling_time_mean    365514 non-null  float64 
 9   dwelling_time_median  365514 non-null  float64 
 10  dwelling_time_std     365514 non-null  float64 
 11  dwelling_time_skew    365514 non-null  float64 
 12  sd_current_min        365514 non-null  float64 
 13  sd_current_max        365514 non-null  float64 
 14  sd_current_mean       365514 non-nul

In [28]:
heatmap(df,figsize=(14,14))

NameError: name 'heatmap' is not defined

## Train-test split

In [61]:
splitter = GroupShuffleSplit(test_size=.20, n_splits=2, random_state = 7)
split = splitter.split(df, groups = df['gene_id'])
train_inds, test_inds = next(split)

train = df.iloc[train_inds]
test = df.iloc[test_inds]

In [62]:
y_train = train['label']
X_train = train.drop(['label'], axis = 1)

y_test = test['label']
X_test = test.drop(['label'], axis = 1)

In [63]:
train_geneids = train['gene_id'].unique()
test_geneids = test['gene_id'].unique()

In [64]:
# check if any gene id in train set is present in test set
np.in1d(train_geneids, test_geneids).any()

False

In [65]:
y_train.value_counts()

0    290578
1      4409
Name: label, dtype: int64

In [66]:
y_train.value_counts()/len(y_train)

0    0.985054
1    0.014946
Name: label, dtype: float64

98.5% of training data consists of data with no m6a modification. Class imbalance!

In [67]:
scaler = StandardScaler()
scaled_numfeats_train = pd.DataFrame(scaler.fit_transform(X_train[num_cols_names]), 
                                     columns=num_cols_names, index= X_train.index)

In [68]:
#Integrate scaled values to the training set
for col in num_cols_names:
    X_train[col] = scaled_numfeats_train[col]

In [69]:
scaled_numfeats_test = pd.DataFrame(scaler.fit_transform(X_test[num_cols_names]), 
                                     columns=num_cols_names, index= X_test.index)

In [70]:
#Integrate scaled values to the test set
for col in num_cols_names:
    X_test[col] = scaled_numfeats_test[col]

In [71]:
cat_col_index = [df.columns.get_loc(c) for c in nonum_cols_names if c in df]

In [72]:
X_train.head()

,gene_id,transcript_id,transcript_position,relative_position,fivemers,dwelling_time_min,dwelling_time_max,dwelling_time_mean,dwelling_time_median,dwelling_time_std,...,mean_current_skew,order_1,order_2,order_3,order_4,order_5,count_A,count_C,count_G,count_T
0,ENSG00000000003,ENST00000373020,511,1,ATAAC,0.666495,-0.857111,-0.306133,-0.390179,-0.151926,...,0.881674,A,T,A,A,C,0.941099,-0.632728,-1.248174,0.649979
1,ENSG00000000003,ENST00000373020,512,2,TAACT,-0.048135,2.247709,2.037172,0.745165,4.316094,...,0.035589,T,A,A,C,T,-0.194667,-0.632728,-1.248174,2.190504
2,ENSG00000000003,ENST00000373020,513,3,AACTC,0.666495,-0.325463,0.440683,0.182373,0.450893,...,-3.458436,A,A,C,T,C,-0.194667,1.277715,-1.248174,0.649979
3,ENSG00000000003,ENST00000373020,688,1,TAAAC,-0.048135,0.312513,1.256141,0.683355,1.614084,...,-1.223034,T,A,A,A,C,0.941099,-0.632728,-1.248174,0.649979
4,ENSG00000000003,ENST00000373020,689,2,AAACA,-0.048135,-0.750781,-0.255522,-0.110410,-0.386196,...,-0.793546,A,A,A,C,A,2.076866,-0.632728,-1.248174,-0.890546


In [73]:
cat_col_index

[27, 25, 24, 4, 26, 28, 3]

In [74]:
X_train1 = X_train.drop(columns=['gene_id', 'transcript_id'])
X_train1.head(5)

,transcript_position,relative_position,fivemers,dwelling_time_min,dwelling_time_max,dwelling_time_mean,dwelling_time_median,dwelling_time_std,dwelling_time_skew,sd_current_min,...,mean_current_skew,order_1,order_2,order_3,order_4,order_5,count_A,count_C,count_G,count_T
0,511,1,ATAAC,0.666495,-0.857111,-0.306133,-0.390179,-0.151926,-0.712066,-0.410225,...,0.881674,A,T,A,A,C,0.941099,-0.632728,-1.248174,0.649979
1,512,2,TAACT,-0.048135,2.247709,2.037172,0.745165,4.316094,0.716984,-0.161440,...,0.035589,T,A,A,C,T,-0.194667,-0.632728,-1.248174,2.190504
2,513,3,AACTC,0.666495,-0.325463,0.440683,0.182373,0.450893,-0.302362,-0.861769,...,-3.458436,A,A,C,T,C,-0.194667,1.277715,-1.248174,0.649979
3,688,1,TAAAC,-0.048135,0.312513,1.256141,0.683355,1.614084,-0.386723,-0.559496,...,-1.223034,T,A,A,A,C,0.941099,-0.632728,-1.248174,0.649979
4,689,2,AAACA,-0.048135,-0.750781,-0.255522,-0.110410,-0.386196,-1.004085,-0.236075,...,-0.793546,A,A,A,C,A,2.076866,-0.632728,-1.248174,-0.890546


In [75]:
y_train1 = y_train.drop(columns=['gene_id', 'transcript_id'])
y_train1.head(5)

0    0
1    0
2    0
3    0
4    0
Name: label, dtype: int64

In [76]:
sm = SMOTENC(categorical_features=cat_col_index, random_state=123, sampling_strategy=.6)
X_trainres, y_trainres = sm.fit_resample(X_train1, y_train1)
# probably need to one-hot :') 

ValueError: could not convert string to float: 'ATAAC'